In [1]:
from google.colab import drive

drive.mount("/content/drive")
DRIVE_ROOT = "/content/drive/MyDrive"

Mounted at /content/drive


In [2]:
%%capture
!pip install faiss-gpu-cu12 datasets optimum transformers

# Create the faiss index

In [3]:
import faiss
from tqdm.auto import tqdm
from pathlib import Path
import numpy as np
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
from datasets import load_from_disk

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [4]:
[f"cuda:{i}" for i in range(torch.cuda.device_count())]

[]

In [ ]:
wikipedia_path = Path(f"{DRIVE_ROOT}/Colab Notebooks/input/Kaggle - LLM Science Exam")
embedding_size = 384
batch_size = 256
max_length = 512
checkpoint = "BAAI/bge-small-en-v1.5"
embedding_size = 384

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint).cuda().half()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

In [ ]:
def transform(batch):
    batch["text"] = ["Represent this sentence for searching relevant passages: " + x for x in batch["text"]]

    tokens = tokenizer(
        batch["text"],
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )

    return tokens.to("cuda")

In [ ]:
# Create faiss index, it will use the same index as wikipedia_index (not the "id", but the row index)
faiss_index = faiss.IndexFlatL2(embedding_size)

In [ ]:
# Create dataset and dataloader
dataset = load_from_disk(wikipedia_path)
dataset.set_transform(transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Compute embeddings
outputs = np.zeros((len(dataset), embedding_size), dtype=np.float16)
with torch.inference_mode():
    for i, batch in tqdm(enumerate(dataloader), leave=False, total=len(dataloader)):
        embeddings = model(**batch).pooler_output
        embeddings = F.normalize(embeddings, p=2, dim=1)
        outputs[batch_size * i:batch_size * (i + 1)] = embeddings.detach().cpu().numpy()

  0%|          | 0/8209 [00:00<?, ?it/s]

In [ ]:
# Add embeddings to faiss index (it will use the same index as wiki_2023_index.parquet)
faiss_index.add(outputs)
faiss.write_index(faiss_index, str(wikipedia_path / f"faiss_index_{checkpoint.split('/')[-1]}.index"))

# Platypus2-70B + Wikipedia RAG

In [ ]:
import gc
import logging
from time import time
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
from threading import Condition
import ctypes
from functools import partial

import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# For RAG
import faiss
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset

NUM_TITLES = 5  # 검색된 문서 개수
MAX_SEQ_LEN = 512
MODEL_NAME = "BAAI/bge-small-en-v1.5"

# For LLM
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, AutoModel
from accelerate import init_empty_weights
from accelerate.utils.modeling import set_module_tensor_to_device
from safetensors.torch import load_file
from optimum.bettertransformer import BetterTransformer

N_BATCHES = 5
MAX_LENGTH = 4096
MAX_CONTEXT = 1200
# With NUM_TITLES = 5, the median lenght of a context if 1100 tokens (Q1: 900, Q3: 1400)

In [ ]:
# Function to clean RAM & VRAM
def clean_memory():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

In [ ]:
# Load data
# df = pd.read_csv(f"{DRIVE_ROOT}/Colab Notebooks/input/Kaggle - LLM Science Exam/test.csv", index_col="id")
df = pd.read_csv(f"{DRIVE_ROOT}/Colab Notebooks/input/Kaggle - LLM Science Exam/train.csv", index_col="id")

IS_TEST_SET = True  # len(df) != 200
N_BATCHES = 1

## Wikipedia RAG

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
class SentenceTransformer:
    def __init__(self, checkpoint, device):
        self.device = device
        self.checkpoint = checkpoint
        self.model = AutoModel.from_pretrained(checkpoint).to(self.device).half()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def transform(self, batch):
        tokens = self.tokenizer(
            batch["text"],
            truncation=True,
            padding=True,
            return_tensors="pt",
            max_length=MAX_SEQ_LEN,
        )
        return tokens.to(self.device)

    def get_dataloader(self, sentences, batch_size=32):
        sentences = ["Represent this sentence for searching relevant passages: " + x for x in sentences]
        dataset = Dataset.from_dict({"text": sentences})
        dataset.set_transform(self.transform)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        return dataloader

    def encode(self, sentences, show_progress_bar=False, batch_size=32):
        dataloader = self.get_dataloader(sentences, batch_size=batch_size)
        pbar = tqdm(dataloader) if show_progress_bar else dataloader

        embeddings = []
        for batch in pbar:
            with torch.no_grad():
                e = self.model(**batch).pooler_output
                e = F.normalize(e, p=2, dim=1)
                embeddings.append(e.detach().cpu().numpy())
        embeddings = np.concatenate(embeddings, axis=0)
        return embeddings

In [ ]:
model = SentenceTransformer(MODEL_NAME, device=device)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
if IS_TEST_SET:
    start = time()
    print(f"Starting prompt embedding, t={time() - start:.1f}s")
    model = SentenceTransformer(MODEL_NAME, device=device)

    # Get embeddings of prompts
    f = lambda row : " ".join([row["prompt"], row["A"], row["B"], row["C"], row["D"], row["E"]])
    inputs = df.apply(f, axis=1).values  # better results than prompt only
    prompt_embeddings = model.encode(inputs, show_progress_bar=True)

    # Search closest sentences in the wikipedia index
    print(f"Loading faiss index, t={time() - start:.1f}s")
    faiss_index = faiss.read_index(f"{DRIVE_ROOT}/Colab Notebooks/input/Kaggle - LLM Science Exam/faiss_index_bge-small-en-v1.5.index")
    # faiss_index = faiss.index_cpu_to_all_gpus(faiss_index)  # OOM 발생 시 주석, CPU에서도 충분히 빠름

    print(f"Starting text search, t={time() - start:.1f}s")
    search_index = faiss_index.search(np.float32(prompt_embeddings), NUM_TITLES)[1]

    print(f"Starting context extraction, t={time() - start:.1f}s")
    dataset = load_from_disk("/content/drive/MyDrive/Colab Notebooks/input/Kaggle - LLM Science Exam")
    for i in range(len(df)):
        df.loc[i, "context"] = "-" + "\n-".join([dataset[int(j)]["text"] for j in search_index[i]])

    # Free memory
    faiss_index.reset()
    del faiss_index, prompt_embeddings, model, dataset
    clean_memory()
    print(f"Context added, t={time() - start:.1f}s")

Starting prompt embedding, t=0.0s


  0%|          | 0/7 [00:00<?, ?it/s]

Loading faiss index, t=2.9s
Starting text search, t=68.0s
Starting context extraction, t=74.2s
Context added, t=83.5s


## Run Platypus2-70B

In [ ]:
import kagglehub

source_dirs = [
    kagglehub.dataset_download("janderchu/platypus2-chuhac2-part1"),
    kagglehub.dataset_download("janderchu/platypus2-chuhac2-part2"),
    kagglehub.dataset_download("janderchu/platypus2-chuhac2-part3")
]

100%|██████████| 50.8G/50.8G [07:45<00:00, 117MB/s]

Extracting files...


100%|██████████| 50.5G/50.5G [07:42<00:00, 117MB/s]

Extracting files...


In [ ]:
from transformers.utils import TRANSFORMERS_CACHE

source_dirs = [
    "/content/drive/MyDrive/Colab Notebooks/model/platypus2-chuhac2/platypus2-chuhac2-part1/versions/1",
    "/content/drive/MyDrive/Colab Notebooks/model/platypus2-chuhac2/platypus2-chuhac2-part2/versions/1",
    "/content/drive/MyDrive/Colab Notebooks/model/platypus2-chuhac2/platypus2-chuhac2-part3/versions/3",
]

# 분할 모델 병합 경로
checkpoint_path = Path(f"{TRANSFORMERS_CACHE}/models--janderchu--platypus2-chuhac2")
checkpoint_path.mkdir(exist_ok=True, parents=True)

# 병합 시 용량 절약을 위해 shutil을 사용하지 않고 symlink로 대체
for source_dir_path in map(Path, source_dirs):
    for path in source_dir_path.glob("*"):
        (checkpoint_path / path.name).symlink_to(path)

FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/Colab Notebooks/model/platypus2-chuhac2/platypus2-chuhac2-part1/versions/1/model.embed_tokens.safetensors' -> '/root/.cache/huggingface/hub/models--janderchu--platypus2-chuhac2/model.embed_tokens.safetensors'

In [ ]:
class WeightsLoader:
    """
    Thread-safe class to load the weights of the model
    The weights are loaded in the background and can be accessed with get_state_dict().
    All devices must call set_state_dict() before the weights are loaded.
    """

    def __init__(self, checkpoint_path, devices):
        self.checkpoint_path = Path(checkpoint_path)
        # 각 디바이스 상태 어떤 레이어를 로드했는지 저장
        self.states = {device: None for device in devices}
        # 현재 로드된 가중치 저장
        self.state_dict = None
        # 스레드 동기화를 위한 Condition 변수
        self.condition = Condition()

    def get_state_dict(self, device):
        """
        특정 디바이스가 가중치를 반환받기 위해 호출하는 메서드.
        스레드는 가중치가 준비되길 기다리다가 준비되면 로드된 가중치를 받아온다.
        """
        with self.condition:
            while self.states[device] is not None:
                self.condition.wait()

            result = self.state_dict
            self.states[device] = None

            if not any(self.states.values()):
                self.condition.notify_all()
        return result

    def set_state_dict(self, layer_name, device):
        """
        각 디바이스가 로드할 레이어 이름을 지정하고,
        모든 디바이스가 특정 레이어에 대해 호출되면 해당 레이어 가중치를 한 번만 로드한다.
        """
        with self.condition:
            self.states[device] = layer_name
            # 모든 디바이스가 어떤 레이어를 요청했는지 확인
            if all(self.states.values()):
                # 모든 디바이스가 같은 레이어를 요청했는지 검증
                assert len(set(self.states.values())) == 1, "All devices should load the same layer"
                self.state_dict = load_file(self.checkpoint_path / (layer_name + ".safetensors"), device="cpu")
                # 모든 디바이스의 상태를 None으로 초기화해 가중치 로드 완료됨을 표시하고 notify 한다.
                for d in self.states:
                    self.states[d] = None
                self.condition.notify_all()

In [ ]:
# Class for shared llama
class SharedLlama:
    def __init__(self, checkpoint_path, weights_loader, device="cuda:0", dtype=torch.float16):
        """
        Shared version of LlamaForCausalLM : the model is splitted into layer shards to reduce GPU memory usage.

        During the forward pass, the inputs are processed layer by layer, and the GPU memory is freed after each layer.

        To avoid loading the layers multiple times, we could save all the intermediate activations in RAM,
        but as Kaggle accelerators have more GPU memory then CPU, we simply batch the inputs and keep them on the GPU

        Parameters
        ----------
        checkpoint_path : str or Path
            path to the checkpoint
        weights_loader : WeightsLoader
            object to load the weights
        device : str, optional
            device, by default "cuda:0"
        dtype : torch.dtype, optional
            dtype, by default torch.float16
        """

        # Save parameters
        self.checkpoint_path = Path(checkpoint)
        self.weights_loader = weights_loader
        self.device = device
        self.dtype = dtype

        # Create model
        self.config = AutoConfig.from_pretrained(checkpoint_path)
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "right"
        self.init_model()
        self.layer_names = ["model.embed_tokens"] + [f"model.layers.{i}" for i in range(len(self.model.model.layers))] + ["model.norm", "value_head"]

    def init_model(self):
        # init_empty_weights()로 메모리 사용을 최소화하며 모델의 구조만 먼저 생성한다.
        with init_empty_weights():
            self.model = AutoModelForCausalLM.from_config(self.config)
            # 원래 출력차원인 32k를 8로 줄인 새로운 선형 레이어를 언어 모델의 헤드로 대체한다.
            self.model.lm_head = torch.nn.Linear(8192, 8, bias=False)  # originally 32k
            self.model.eval()
            # 모델을 최적화된 transformer 구조로 변환한다. flash attention과 같은 최적화 기법을 사용해 추론 속도 개선 가능
            self.model = BetterTransformer.transform(self.model)  # enable flash attention
            # 입력 임베딩과 출력 프로젝션 같은 공유가 가능한 가중치들을 연결한다.
            self.model.tie_weights()

        self.layers = [self.model.model.embed_tokens] + list(self.model.model.layers) + [self.model.model.norm, self.model.lm_head]

        # Move buffers to device
        # 모델 내부의 파라미터는 아니지만 학습이나 추론에 필요한 상태(배치 정규화의 이동 평균 등)를 저장하는 버퍼 반환
        # 각 버퍼를 지정한 디바이스로 옮기고, 데이터 타입을 변환
        for buffer_name, buffer in self.model.named_buffers():
            set_module_tensor_to_device(self.model, buffer_name, self.device, value=buffer, dtype=self.dtype)

    def load_layer_to_cpu(self, layer_name):
        self.weights_loader.set_state_dict(layer_name, self.device)
        state_dict = self.weights_loader.get_state_dict(self.device)
        if "value_head.weight" in state_dict:
            state_dict = {"lm_head.weight": state_dict["value_head.weight"]}
        return state_dict

    def move_layer_to_device(self, state_dict):
        for param_name, param in state_dict.items():
            assert param.dtype != torch.int8, "int8 not supported (need to add fp16_statistics)"
            set_module_tensor_to_device(self.model, param_name, self.device, value=param, dtype=self.dtype)

    def __call__(self, inputs):
        # Reboot the model to make sure buffers are loaded and memory is clean
        del self.model
        clean_memory()
        self.init_model()

        # Send batch to device
        batch = [(prefix.to(self.device), suffix.to(self.device)) for prefix, sufix in inputs]
        # 첫
        n_suffixes = len(batch[0][1])
        suffix_eos = [(suffix != self.tokenizer.pad_token_id).sum(1) - 1 for _, suffix in inputs]

        # Create attention mask for the largest input, and position ids to use KV cache
        attention_mask = torch.ones(MAX_LENGTH, MAX_LENGTH)
        attention_mask = attention_mask.triu(diagonal=1)[None, None, ...] == 0
        attention_mask = attention_mask.to(self.device)
        position_ids = torch.arange(MAX_LENGTH, dtype=torch.long, device=self.device)[None, :]

        with ThreadPoolExecutor() as executor, torch.inference_mode():
            # Load first layer
            future = executor.submit(self.load_layer_to_cpu, "model.embed_tokens")

            for i, (layer_name, layer) in tqdm(enumerate(zip(self.layer_names, self.layers)), desc=self.device, total=len(self.layers)):
                # Load current layer and prepare next layer
                state_dict = future.result()
                if (i + 1) < len(self.layer_names):
                    future = executor.submit(self.load_layer_to_cpu, self.layer_names[i + 1])
                self.move_layer_to_device(state_dict)

                # Run layer
                for j, (prefix, suffix) in enumerate(batch):
                    if layer_name == "model.embed_tokens":
                        batch[j] = (layer(prefix), layer(suffix))
                    elif layer_name == "model.norm":
                        # Only keep the last token at this point
                        batch[j] = (None, layer(suffix[torch.arange(n_suffixes), suffix_eos[j]][:, None]))
                    elif layer_name == "value_head":
                        batch[j] = layer(suffix)[:, 0].mean(1).detach().cpu().numpy()
                    else:
                        # Run prefix
                        len_p, len_s = prefix.shape[1], suffix.shape[1]
                        new_prefix, (k_cache, v_cache) = layer(prefix, use_cache=True, attention_mask=attention_mask[:, :, -len_p:, -len_p:])

                        # Run suffix
                        pos = position_ids[:, len_p:len_p + len_s].expand(n_suffixes, -1)
                        attn = attention_mask[:, :, -len_s:, -len_p - len_s:].expand(n_suffixes, -1, -1, -1)
                        kv_cache = (k_cache.expand(n_suffixes, -1, -1, -1), v_cache.expand(n_suffixes, -1, -1, -1))
                        new_suffix = layer(suffix, past_key_value=kv_cache, position_ids=pos, attention_mask=attn)[0]
                        batch[j] = (new_prefix, new_suffix)

                # Remove previous layer from memory (including buffers)
                layer.to("meta")
                clean_memory() # proposed by CPMP

        # Get scores
        return batch

In [ ]:
# Run model on the 2 GPUs

def get_tokens(row, tokenizer):
    system_prefix = "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\nContext:\n{context}"
    instruction = "Your task is to analyze the question and answer below. If the answer is correct, respond yes, if it is not correct respond no. As a potential aid to your answer, background context from Wikipedia articles is at your disposal, even if they might not always be relevant."

    # max length : MAX_LENGTH
    prompt_suffix = [f"{row[letter]}\n\n### Response:\n" for letter in "ABCDE"]
    suffix = tokenizer(prompt_suffix, return_tensors="pt", return_attention_mask=False, truncation=True, max_length=MAX_LENGTH, padding=True)["input_ids"][:, 1:]

    # max length : max(0, MAX_LENGTH - len(suffix))
    prompt_question = f"\nQuestion: {row['prompt']}\nProposed answer: "
    question = tokenizer(prompt_question, return_tensors="pt", return_attention_mask=False, truncation=True, max_length=max(0, MAX_LENGTH - suffix.shape[1]))["input_ids"][:, 1:]

    # max length : min(MAX_CONTEXT, max(0, MAX_LENGTH - len(suffix) - len(question)))
    prompt_context = system_prefix.format(instruction=instruction, context=row["context"])
    max_length = min(MAX_CONTEXT, max(0, MAX_LENGTH - question.shape[1] - suffix.shape[1]))
    context = tokenizer(prompt_context, return_tensors="pt", return_attention_mask=False, truncation=True, max_length=max_length)["input_ids"]

    prefix = torch.cat([context, question], dim=1)
    return prefix, suffix

def run_model(device, df, weights_loader):
    model = ShardedLlama(checkpoint_path, weights_loader, device=device)
    f = partial(get_tokens, tokenizer=model.tokenizer)
    inputs = df.apply(f, axis=1).values
    batches = np.array_split(inputs, N_BATCHES)
    outputs = []
    for i, batch in enumerate(batches):
        outputs += model(batch)
    return outputs

# Run model
if IS_TEST_SET:
    devices = [f"cuda:{i}" for i in range(torch.cuda.device_count())]
    weights_loader = WeightsLoader(checkpoint_path, devices)
    f = partial(run_model, weights_loader=weights_loader) # added by treesky
    with ThreadPoolExecutor() as executor:
        outputs = list(executor.map(f, devices, np.array_split(df, 2)))
        outputs = sum(outputs, [])

    # Save results
    n = len(df)
    for i, scores in enumerate(outputs):
        top3 = np.argsort(scores)[::-1]
        df.loc[i, "prediction"] = " ".join(["ABCDE"[j] for j in top3])

    # Display performances if train set is used
    if "answer" in df.columns:
        for i in range(n):
            df.loc[i, "top_1"] = df.loc[i, "prediction"][0]
            df.loc[i, "top_2"] = df.loc[i, "prediction"][2]
            df.loc[i, "top_3"] = df.loc[i, "prediction"][4]

        top_i = [(df[f"top_{i}"] == df["answer"]).sum() for i in [1, 2, 3]]
        print(f"top1 : {top_i[0]}/{n}, top2 : {top_i[1]}/{n}, top3 : {top_i[2]}/{n} (total={sum(top_i)} / {n})")
        print(f"Accuracy: {100*top_i[0]/n:.1f}%, map3: {100*(top_i[0] + top_i[1]*1/2 + top_i[2]*1/3).sum()/n:.1f}%")
else:
    df["prediction"] = "A B C"

df[["prediction"]].to_csv("submission.csv")